# Human Failure Representation

This paper covers the exploration and analysis of error producing conditions by a human operator in the rover model.

In [1]:
from rover_model_human import Rover, gen_params
from rover_model import plot_map, plot_trajectories
import fmdtools.analyze as an
import fmdtools.sim.propagate as prop
from fmdtools.sim.approach import SampleApproach, NominalApproach

import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'rover_model_human'

In [2]:
prop

Visualizing Rover Structure

In [3]:
mdl = Rover()

In [4]:
#%matplotlib qt
#an.graph.set_pos(mdl, gtype='fxnflowgraph')

In [5]:
#%matplotlib inline

In [6]:
mdl = Rover()
fig, ax = an.graph.show(mdl, gtype='fxnflowgraph')

In [7]:
fig.savefig("rover_structure.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

Visualizing Action Sequence Graph for the Controller

In [8]:
mdl.fxns['Controller'].action_graph

In [9]:
#%matplotlib qt
#an.graph.set_pos(mdl.fxns['Controller'].flow_graph)

In [10]:
#%matplotlib inline

In [11]:
act_pos={'Press': [0.942, 0.076], 'ComprehendOut': [0.302, 0.452], 'SeeOut': [0.028, 0.732], 
             'Project': [0.499, 0.46], 'PressOut': [0.958, -0.07], 'Reach': [0.797, 0.175], 
             'ProjectOut': [0.482, 0.302], 'ReachOut': [0.806, 0.041], 'Comprehend': [0.317, 0.595], 
             'Look': [0.038, 0.877], 'Decide': [0.667, 0.308], 'DecideOut': [0.652, 0.189], 
             'Signal': [0.721, 0.736], 'Percieve': [0.165, 0.738], 'PercieveOut': [0.102, 0.594], 
             'Video': [0.418, 0.889], 'Stimulus': [0.026, 1.034], 'LocalPSF': [0.717, 0.484], 
             'GlobalPSF': [0.521, 0.836]}

In [12]:
%matplotlib qt
an.graph.set_pos(mdl.fxns['Controller'].flow_graph, pos=act_pos)

In [13]:
%matplotlib inline

In [14]:
fig = mdl.fxns['Controller'].show_ASG(pos=act_pos, with_cond_labels=False)

In [15]:
fig.savefig("action_graph.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

## Testing behavioral simulation

In [16]:
mdl = Rover()

Execution order

In [17]:
order_fig = an.graph.exec_order(mdl, gtype='fxnflowgraph')

Performance on turn

In [18]:
mdl = Rover(params=gen_params('turn', start=5))
endresults, mdlhist = prop.nominal(mdl)
plot_map(mdl, mdlhist)

Performance on sine map

In [ ]:
mdl = Rover(params=gen_params('sine', amp=2, wavelength=30))
endresults, mdlhist = prop.nominal(mdl)
plot_map(mdl, mdlhist)

In [ ]:
phases, modephases = mdlhist.get_modephases()

In [ ]:
phase_fig = an.plot.phases(phases, modephases=modephases, title_padding=-0.05, v_padding=0.5)

## Testing Faults

In [ ]:
app=SampleApproach(mdl, faults='Controller', phases={'drive':phases['Operations']['drive']})

In [ ]:
app.rates

In [ ]:
app.times

In [ ]:
endclasses,mdlhists = prop.approach(mdl, app)

In [ ]:
fig = plt.figure(figsize=(5.25,3.5))
fig = plot_trajectories(mdlhists, app=app, faultlabel='Faulty Scenarios', title='', mode_trunc=len('Controller'), mode_trunc_end=4, show_labels=False, xlim=(-1,65), ylim=(-12,12))

In [ ]:
fig

In [ ]:
fig.savefig("rover_map.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

Below shows the impact of each fault:

In [ ]:
new_fault_tab = an.tabulate.result_summary_fmea(endclasses, mdlhists)
new_fault_tab = new_fault_tab.sort_values('end_dist', ascending=False)

new_fault_tab 

In [ ]:
fault_tab = new_fault_tab[['degraded functions','degraded flows', 'classification', 'end_dist']]
fault_tab

In [ ]:
fault_tab

In [ ]:
fault_tab = fault_tab[:15]

In [ ]:
print(fault_tab.to_latex())

As shown, most modes now have an effect.

## Examining Performance Shaping Factors

In [ ]:
nomapp = NominalApproach()
nomapp.add_param_ranges(gen_params,'PSFs', 'sine', fatigue=(0,11,1),attention=(0,11,1), amp=4)

In [ ]:
endresults, mdlhists = prop.nominal_approach(mdl, nomapp)

In [ ]:

fig = an.plot.nominal_vals_2d(nomapp, endresults, 'fatigue', 'attention', title="", legendloc="upper left", figsize=(3.5,3.5))

In [ ]:
fig.savefig("nominal_psfs.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

In [ ]:
mdl = Rover(params=gen_params('sine', amp=2, fatigue=0, wavelength=30))
endresults, mdlhist = prop.one_fault(mdl, 'Operations', 'no_con', time=15)

In [ ]:
endresults['faults']

In [ ]:
traj_fig = plot_trajectories(mdlhist, faultalpha=0.9)

In [ ]:
fig = an.plot.mdlhistvals(mdlhist, time=15, fxnflowvals={'GlobalPSF':['fatigue','attention'], 'Ground':['ang', 'vel']}, legend=False)

In [ ]:
mdlhist['nominal'].keys()

In [ ]:
faulthists = {}
faulttimes = [j for j in range(0,30,5)]
for j in faulttimes:
    for i in range(10):
        np.random.seed()
        mdl = Rover(params=gen_params('turn', start=10, fatigue=0))
        endresults, mdlhist = prop.one_fault(mdl, 'Operations', 'no_con', time=j)
        faulthists[str(i+j)] = mdlhist['faulty']

In [ ]:
fig = plt.figure(figsize=(4.5,3))
fig = plot_trajectories(faulthists,mdlhist['nominal'], faultalpha=0.5, show_labels=False, xlim=(-1,45), ylim=(-5,22))
for t in faulttimes:
    plt.annotate("t="+str(t), (mdlhist['nominal']['flows']['Ground']['x'][t], mdlhist['nominal']['flows']['Ground']['y'][t]))
plt.scatter(mdlhist['nominal']['flows']['Ground']['x'][faulttimes], mdlhist['nominal']['flows']['Ground']['y'][faulttimes], marker='X', color='black', label='fault time')
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

In [ ]:
fig.savefig("nocon_trajectories.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

In [ ]:
plothists = faulthists
plothists['nominal'] = mdlhist['nominal']

In [ ]:
fig, ax = an.plot.mdlhists(plothists, comp_groups={'no_con':faulthists.keys(),'nominal':['nominal']}, fxnflowvals={'GlobalPSF':['attention']}, cols=1, time_slice=faulttimes, indiv_kwargs={'no_con':{'alpha':0.5, 'color':'red'}, 'nominal':{'color':'blue'}}, time_slice_label='fault time', figsize=(4.5,3),  ylabels={('GlobalPSF','attention'):'attention parameter'})

In [ ]:
fig.savefig("nocon_attention.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

In [ ]:
faulthists = {}
faulttimes = [j for j in range(0,30,5)]
for j in faulttimes:
    for i in range(10):
        np.random.seed()
        mdl = Rover(params=gen_params('sine', amp=2, fatigue=0, wavelength=30))
        endresults, mdlhist = prop.one_fault(mdl, 'Operations', 'no_con', time=j)
        faulthists[str(i+j)] = mdlhist['faulty']

In [ ]:
fig = plt.figure(figsize=(4.5,3))
fig = plot_trajectories(faulthists,mdlhist['nominal'], faultalpha=0.3, show_labels=False, xlim=(-1,32), ylim=(-8,8))
for t in faulttimes:
    plt.annotate("t="+str(t), (mdlhist['nominal']['flows']['Ground']['x'][t], mdlhist['nominal']['flows']['Ground']['y'][t]))
plt.scatter(mdlhist['nominal']['flows']['Ground']['x'][faulttimes], mdlhist['nominal']['flows']['Ground']['y'][faulttimes], marker='X', color='black', label='fault time')
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

In [ ]:
fig.savefig("nocon_trajectories_sine.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

In [ ]:
plothists = faulthists
plothists['nominal'] = mdlhist['nominal']

In [ ]:
fig, ax = an.plot.mdlhists(plothists, comp_groups={'no_con':faulthists.keys(),'nominal':['nominal']}, fxnflowvals={'GlobalPSF':['attention']}, cols=1, time_slice=faulttimes, indiv_kwargs={'no_con':{'alpha':0.5, 'color':'red'}, 'nominal':{'color':'blue'}}, time_slice_label='fault time', figsize=(4.5,3), ylabels={('GlobalPSF','attention'):'attention parameter'})

In [ ]:
fig.savefig("nocon_attention_sine.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)